In [0]:
#Tensorflow Import and GPU recognition
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15591853187351562361, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5560800193318146679
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15652068363312849896
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648777152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3583342166604333995
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
!unzip '/content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip' -d '/content'

Archive:  /content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip
replace /content/fruits-360_dataset/fruits-360/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
#Importing Keras and other useful libs
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from glob import glob
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [0]:
#Defining dataset path and saving it for future usage
path = '/content/fruits-360_dataset/fruits-360'
os.listdir(path)

['LICENSE', 'Test', 'Training', 'papers', 'test-multiple_fruits', 'readme.md']

In [0]:
#Number of pictures in Training folder
training_files = glob(os.path.join(path,'Training', '*/*.jpg'))
image_num = len(training_files)
print("Number of Images: ",image_num)

Number of Images:  60498


In [0]:
#Number of pictures in Test Folder
testing_files = glob(os.path.join(path, 'Test', '*/*.jpg'))
img_num = len(testing_files)
print("Number of Images: ", img_num)

Number of Images:  20622


In [0]:
#Print category details
image_count = []
class_names = []
print('{:18s}'.format('Class'), end='')
print('Count:')
print('-'*24)
for folder in os.listdir(os.path.join(path,'Training')):
  folder_count = len(os.listdir(os.path.join(path,'Training',folder)))
  image_count.append(folder_count)
  class_names.append(folder)
  print('{:20s}'.format(folder), end='')
  print(folder_count)
print('-'*24)
print('Number of Classes:', len(class_names))
print('Average number of images per Class: ', np.array(image_count).mean())

Class             Count:
------------------------
Grape White 2       490
Mango               490
Raspberry           490
Beetroot            450
Mulberry            492
Tomato 1            738
Grape Pink          492
Kumquats            490
Nut Forest          218
Apple Pink Lady     456
Kohlrabi            471
Tamarillo           490
Apple Crimson Snow  444
Grape White         490
Banana Red          490
Mango Red           426
Grape Blue          984
Ginger Root         99
Cantaloupe 1        492
Melon Piel de Sapo  738
Maracuja            490
Pineapple Mini      493
Nectarine           492
Pear Kaiser         300
Apple Red 1         492
Pineapple           490
Pitahaya Red        490
Kiwi                466
Pear Forelle        702
Limes               490
Lemon               492
Cocos               490
Cherry Rainier      738
Huckleberry         490
Cherry Wax Yellow   492
Tomato Maroon       367
Granadilla          490
Dates               490
Pear Red            666
Tomato 3       

In [0]:
#Definitive paths
train_out_path = os.path.join(path,'Training')
test_out_path = os.path.join(path, 'Test')
print(train_out_path)
print(test_out_path)

/content/fruits-360_dataset/fruits-360/Training
/content/fruits-360_dataset/fruits-360/Test


In [0]:
#2 Approach: Inception v3 pre-trained
from keras_applications.vgg16 import VGG16
from keras_applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras import optimizers
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras import regularizers
from keras.layers.convolutional import *
from keras.models import Model

In [0]:
#Implementing some augmentation to avoid overfitting on the training generator
train_datagenerator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
train_and_val_generator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True)
test_datagenerator = ImageDataGenerator(rescale= 1./255)

In [0]:
#Creating Batches
image_size = (80, 80)
train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed = 20052020)
val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', seed = 20052020)
train_val_batches = train_and_val_generator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, seed = 20052020)
test_batches = test_datagenerator.flow_from_directory(directory=test_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False)

Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 60498 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [0]:
inception_v3_model = applications.InceptionV3(include_top=False, weights='imagenet', input_shape=image_size+(3,))
#Using pretrained
for layer in inception_v3_model.layers:
  layer.trainable=False
#Adding Layers at the top
top = inception_v3_model.output
top = GlobalAveragePooling2D(data_format='channels_last', name='avg_pool')(top)

#Adding Classification layer with softmax activation function(needed for categorical predictions)
predictor = Dense(120 ,activation='softmax', name='predictor')(top)
inception_v3_model = Model(inputs=inception_v3_model.input, outputs=predictor)

In [0]:
inception_v3_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 80, 80, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 39, 39, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 39, 39, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 39, 39, 32)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
#Compiling the model
inception_v3_model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
inception_v3_model.save_weights('Initial weights.h5')

In [0]:
#Training and Validating the model: Round 1
wait = 1
epochs = 15
model_name='Inception v3'
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=wait)

In [0]:
inception_v3_model.fit_generator(train_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches, callbacks=[early_stopping])
inception_v3_model.load_weights('Initial weights.h5')
trainingMdl = inception_v3_model.fit_generator(train_val_batches, epochs=early_stopping.stopped_epoch , verbose=1, shuffle=True, validation_data=val_batches)

Epoch 1/5
1514/1514 [==============================] - 136s 90ms/step - loss: 2.6100 - accuracy: 0.4392 - val_loss: 11.7555 - val_accuracy: 0.0743
Epoch 2/5
1514/1514 [==============================] - 127s 84ms/step - loss: 1.6900 - accuracy: 0.5931 - val_loss: 21.9243 - val_accuracy: 0.0800
Epoch 1/1
1891/1891 [==============================] - 155s 82ms/step - loss: 2.3908 - accuracy: 0.4743 - val_loss: 17.3011 - val_accuracy: 0.0738


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Progetto Advanced Machine Learning/')
from utils import plot_history
history_fig = plot_history(trainingMdl)
history_fig.show()